In [77]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from classification.models.crnn.model import AudioCRNN # this might change depending on import mechanism

import torch
from datasets.audioset import Audioset
from torchvision import transforms
from torch.utils.data import DataLoader

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## CRNN

Zero pads input to max length in batch. Required for Pytorch RNN's.

Note that `fixed_padding=False` is given to Audioset instance and `collate_fn=audio_set_val.pad_seq` is given to Dataloader.

In [79]:
# audio_set_tr = Audioset(split_mode='training', transforms=dataset_transforms)
# audio_set_val = Audioset(split_mode='validation', transforms=dataset_transforms)
audio_set_val = Audioset(split_mode='validation', fixed_padding=False)
# tr_loader = DataLoader(audio_set_tr, batch_size=32)
val_loader = DataLoader(audio_set_val, batch_size=1, collate_fn=audio_set_val.pad_seq)

In [73]:
res_path = '/nfs/students/summer-term-2020/project-4/data/models/crnn.pth'
checkpoint = torch.load(res_path)

model = AudioCRNN(checkpoint['classes'], checkpoint['config'])
model.load_state_dict(checkpoint['state_dict'])
model = model.to(device) 
model = model.eval()

In [80]:
for i, data in enumerate(val_loader):
#     print(batch[2])
    data = [item.to(device) for item in data] # move to gpu
    batch, labels = data[:3], data[-1]
    print(batch[0])
    print(batch[0].shape)
    print(model(batch))
    break

tensor([[-0.2285, -0.2219, -0.1671,  ...,  0.0876,  0.1575,  0.0000]],
       device='cuda:0')
torch.Size([1, 480375])
tensor([[-0.0210, -3.8717]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)


## RecursiveCNN

Zero pads input to max length of dataset.

Note that `fixed_padding=True` is given to Audioset instance.

In [96]:
audio_set_val = Audioset(split_mode='validation', fixed_padding=True)
val_loader = DataLoader(audio_set_val, batch_size=1)

In [82]:
from classification.models.DeepRecursiveCNN import DeepRecursiveCNN

model_state_dict_path = "/nfs/students/summer-term-2020/project-4/data/models/DeepRecursiveCNN.pt"
model = DeepRecursiveCNN()
model.load_state_dict(torch.load(model_state_dict_path))
model = model.eval()
model = model.to(device) 

In [97]:
for i, data in enumerate(val_loader):
#     print(batch[2])
    data = [item.to(device) for item in data] # move to gpu
    batch, labels = data[:3], data[-1]
    print(batch[0])
    print(batch[0].shape)
    print(model(batch))
    break

tensor([[-0.2285, -0.2219, -0.1671,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')
torch.Size([1, 481489])
tensor([[-0.0536, -2.9526]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)
